# Task II: Classical GNNs (Part I) #

This is my attempt at implementing a classical GNN to classify Quark and Gluon jets.

Considering that this is a point cloud dataset, I decided to use a PointNet architecture. This was developed specifically for messy data clouds to be converted into a graph-like structure to perform classification, segmentation, or other data maneuvers. I had never used it before, but the theory behind it made enough sense to give it a try.

This is one of my first data extraction methods when I was trying to understand the data format and representation. I didn't end up using it directly, but it helped me visualize the data storage structure.

In [1]:
import numpy as np

data = np.load('qg_dataset/QG_jets.npz', allow_pickle=True)
DATA_SIZE = 0
for key, value in data.items():
    print(key, value)
    DATA_SIZE = len(value)

X [[[ 2.68769142e-01  3.56903171e-01  4.74138734e+00  2.20000000e+01]
  [ 1.60076377e-01 -2.55609533e-01  4.55022910e+00  2.20000000e+01]
  [ 1.14868731e+00 -6.24380156e-02  4.50385377e+00 -2.11000000e+02]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 1.21266321e+00 -1.12853089e-01  3.04708757e+00 -2.11000000e+02]
  [ 2.40893976e-01 -1.67174886e-02  2.82705667e+00  2.20000000e+01]
  [ 1.02778452e-01 -8.58720522e-02  3.04180579e+00  2.20000000e+01]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 2.16829416e-01 -9.97057017e-01  5.32568913e-01  2.20000000e+01]
  [ 2.31359397e-01 -1.59192211e+00  2.02906587e-01  2.20000000e+01]
  [ 3.41572501e-01 -1.34588077

In [2]:
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader

First extract the data like in Task II. I'm not sure why, but the first time you run the data cell, or any cell with DataLoader, it comes up with a warning. It's fine if you run it again though and doesn't seem to affect the model.

In [3]:
# from torch_geometric.nn import knn_graph
from torch_geometric.transforms import KNNGraph

class JetDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, train=False):
        self.data = np.load(data_path, allow_pickle=True)
        self.samples = []
        self.num_classes = 2
        self.train = train

        for key, value in self.data.items():
            if key == 'X':
                self.samples.extend([(sample, label) for sample, label in zip(value, self.data['y'])])
        
        # 80/20 split for training and testing
        split_index = int(0.8 * len(self.samples))
        if self.train:
            self.samples = self.samples[:split_index]
        else:
            self.samples = self.samples[split_index:]
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample, label = self.samples[idx]
        sample = torch.tensor(sample.transpose(), dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        
        # edge_index = knn_graph(pos, k=6, batch=None, loop=False)
        edge_index = KNNGraph(k=6)
        
        return Data(x=sample, edge_index=edge_index, y=label)


Create a custom JetDataset to properly format the data and split the samples into testing and training (80/20 split). The dictionary of X and y key value pairs is extracted into an array.

In [4]:
class JetDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, train=False):
        self.data = np.load(data_path, allow_pickle=True)
        self.samples = []
        self.num_classes = 2
        self.train = train

        for key, value in self.data.items():
            if key == 'X':
                self.samples.extend([(sample, label) for sample, label in zip(value, self.data['y'])])
        
        # 80/20 split for training and testing
        split_index = int(0.8 * len(self.samples))
        if self.train:
            self.samples = self.samples[:split_index]
        else:
            self.samples = self.samples[split_index:]
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample, label = self.samples[idx]
        sample = torch.tensor(sample.transpose(), dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long) 
        return sample, label

Load the data

In [5]:
import torch_geometric.transforms as T

data_path = 'qg_dataset/QG_jets.npz'
train_dataset = JetDataset(data_path, train=True)
test_dataset = JetDataset(data_path, train=False)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

c:\Users\realc\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [8]:
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing

class PointNetLayer(MessagePassing):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__(aggr='max')

        self.mlp = Sequential(
            Linear(in_channels + 3, out_channels),
            ReLU(),
            Linear(out_channels, out_channels),
        )

    def forward(self,
        h: Tensor,
        pos: Tensor,
        edge_index: Tensor,
    ) -> Tensor:
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self,
        h_j: Tensor,
        pos_j: Tensor,
        pos_i: Tensor,
    ) -> Tensor:

        edge_feat = torch.cat([h_j, pos_j - pos_i], dim=-1)
        return self.mlp(edge_feat)

In [17]:
from torch_geometric.nn import global_max_pool

class PointNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, train_dataset.num_classes)

    def forward(self, pos: Tensor, edge_index: Tensor, batch: Tensor) -> Tensor:

        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        h = global_max_pool(h, batch)

        return self.classifier(h)

Initialize the model and the Adam optimizer and load all the data to be in the correct format to use in PyTorch models.

In [18]:
model = PointNet()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion=torch.nn.CrossEntropyLoss()

train_dataset = JetDataset(data_path=data_path, train=True)
train_dataset.transform = T.Compose([T.SamplePoints(num=128), T.KNNGraph(k=6)])
test_dataset = JetDataset(data_path=data_path, train=False)
test_dataset.transform = T.Compose([T.SamplePoints(num=128), T.KNNGraph(k=6)])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

PointNet(
  (conv1): PointNetLayer()
  (conv2): PointNetLayer()
  (classifier): Linear(in_features=32, out_features=2, bias=True)
)


c:\Users\realc\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


With this particular implementation, I ran into a few difficulties, such as knn_graph imports (some of the knn_graph usage is left in commented code). I was trying to use it to connect the nearest 6 nodes in the dataset to each other to create edges and a sort of interconnected structure, but I either went down the rabbit hole of StackOverflow of trying to make knn_graph compatible with my environment, or the structure of KNNGraph not quite working the way I expected. Overall, this was an interesting problem, but I spent a long time on the formatting bit. This was partially due to the fact that I didn't know what exactly the QG dataset contained (the official website only contained the general format but no sample) and spent a long time looking at what different items printed out, and partially because I am more familiar with graphs in a theoretical sense.

In [13]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        
        edge_index = (data.edge_index[0].long(), data.edge_index[1].long())
        
        logits = model(data.pos, edge_index, data.batch)
        loss = criterion(logits, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs 
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test():
    model.eval()

    total_correct = 0
    for data in test_loader:
        logits = model(data.pos, data.edge_index, data.batch)
        pred = logits.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())

    return total_correct / len(test_loader.dataset)

for epoch in range(1, 51):
    loss = train()
    test_acc = test()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}')



AttributeError: 'list' object has no attribute 'edge_index'